In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'gender_submission.csv', 'test.csv']


In [152]:
df_train = pd.read_csv("../input/train.csv")
df_test = pd.read_csv("../input/test.csv")

In [153]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [154]:
train = df_train

In [155]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [156]:
#complete age with median
train['Age'].fillna(train['Age'].median(), inplace = True)

#complete embarked with mode
train['Embarked'].fillna(train['Embarked'].mode()[0], inplace = True)

In [157]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       891 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [158]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [159]:
train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [160]:
del train['Name']
del train['Ticket']
del train['Cabin']
del train['PassengerId']

In [161]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [162]:
X = train.iloc[:,1:8].values
y = train.iloc[:,0].values

In [163]:
X[0,:]

array([3, 'male', 22.0, 1, 0, 7.25, 'S'], dtype=object)

In [164]:
def encode(X) :
    le_s = LabelEncoder()
    le_e = LabelEncoder()
    X[:,1] = le_s.fit_transform(X[:,1])
    X[:,6] = le_e.fit_transform(X[:,6])
    return X

def OHEncode(X) :
    ohe = OneHotEncoder(categorical_features=[6])
    X = ohe.fit_transform(X).toarray()
    #X = X[:,:] - X[:,[147,150]]
    return X

In [165]:
X = encode(X)
print(X[0,:])

[3 1 22.0 1 0 7.25 2]


In [166]:
X = OHEncode(X)
#X = np.delete(X, 147, 1)     #to escape dummy variable trap
#X = np.delete(X, 149, 1)
X = X[:,1:]
X[0,:]

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:405: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:427: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


array([ 0.  ,  1.  ,  3.  ,  1.  , 22.  ,  1.  ,  0.  ,  7.25])

In [167]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [168]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [172]:
rfr =  RandomForestClassifier (n_estimators=151, max_depth=14,max_features= "log2", random_state = 21)
rfr.fit(X_train,y_train)

ext = ExtraTreesClassifier(n_estimators=151, max_depth=14, max_features= "log2", random_state = 21)
ext.fit(X_train,y_train)

dtr = DecisionTreeClassifier(max_depth=14, max_features= "log2", random_state = 21)
dtr.fit(X_train,y_train)

svc = SVC(kernel = 'rbf',random_state = 21)#, max_features = "log2")
svc.fit(X_train,y_train)

print("Random Forest Model")
print("Train Score {}".format(rfr.score(X_train,y_train)))
print("Test Score {}".format(rfr.score(X_test,y_test)))
print("\n")
print("ExtraTreesRegressor Model")
print("Train Score {}".format(ext.score(X_train,y_train)))
print("Test Score {}".format(ext.score(X_test,y_test)))
print("\n")
print("DecisonTree Model")
print("Train Score {}".format(dtr.score(X_train,y_train)))
print("Test Score {}".format(dtr.score(X_test,y_test)))
print("\n")
print("Support Vector Model")
print("Train Score {}".format(svc.score(X_train,y_train)))
print("Test Score {}".format(svc.score(X_test,y_test)))

Random Forest Model
Train Score 0.9760479041916168
Test Score 0.8699551569506726


ExtraTreesRegressor Model
Train Score 0.9565868263473054
Test Score 0.820627802690583


DecisonTree Model
Train Score 0.9520958083832335
Test Score 0.7937219730941704


Support Vector Model
Train Score 0.8847305389221557
Test Score 0.726457399103139


/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [173]:
# for i in range(150,300):
#     for j in range(0,100):
#         ext = ExtraTreesClassifier(n_estimators=i, max_depth=14, max_features= "log2", random_state = j)
#         ext.fit(X_train,y_train)
        
#         rfr =  RandomForestClassifier (n_estimators=i, max_depth=14,max_features= "log2", random_state = j)
#         rfr.fit(X_train,y_train)

#         dtr = DecisionTreeClassifier(max_depth=14, max_features= "log2", random_state = j)
#         dtr.fit(X_train,y_train)
#         #e = symmetric_mean_absolute_percentage_error(y_test, ext.predict(X_test))
#         #r = symmetric_mean_absolute_percentage_error(y_test, rfr.predict(X_test))
#         #d = symmetric_mean_absolute_percentage_error(y_test, dtr.predict(X_test))
#         e = ext.score(X_test,y_test)
#         r = rfr.score(X_test,y_test)
#         d = dtr.score(X_test,y_test)
#         #print(e,r,d)
#         if max(e,r,d) == e :
#             a = 'ext'
#         elif max(e,r,d) == r :
#             a = 'rfr'
#         else :
#             a = 'dtr'
#         if (max(e,r,d) > 0.86) :
#             print('est = '+str(i)+'|'+'rs = '+str(j)+'|'+str(max(e,r,d))+'|'+a)
#         else :
#             print('working...', end='\r')

Fitting with the complete training set

In [174]:
rfr =  RandomForestClassifier (n_estimators=151, max_depth=14,max_features= "log2", random_state = 21)
rfr.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=14, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=151, n_jobs=1,
            oob_score=False, random_state=21, verbose=0, warm_start=False)

Processing the testing set

In [175]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [176]:
test = df_test
passenger_id = test['PassengerId'].values

In [177]:
del test['Name']
del test['Ticket']
del test['Cabin']
del test['PassengerId']

In [178]:
test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


In [179]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
Pclass      418 non-null int64
Sex         418 non-null object
Age         332 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        417 non-null float64
Embarked    418 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 22.9+ KB


In [180]:
#complete age with median
test['Age'].fillna(test['Age'].median(), inplace = True)

#complete embarked with mode
test['Embarked'].fillna(test['Embarked'].mode()[0], inplace = True)

#complete missing fare with median
test['Fare'].fillna(test['Fare'].median(), inplace = True)

In [181]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
Pclass      418 non-null int64
Sex         418 non-null object
Age         418 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        418 non-null float64
Embarked    418 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 22.9+ KB


In [182]:
X_test = test.values
print(X_test[0,:])

[3 'male' 34.5 0 0 7.8292 'Q']


In [183]:
X_test = encode(X_test)
print(X_test[0,:])

[3 1 34.5 0 0 7.8292 1]


In [184]:
X_test = OHEncode(X_test)
#X = np.delete(X, 147, 1)     #to escape dummy variable trap
#X = np.delete(X, 149, 1)
X_test = X_test[:,1:]
X_test[0,:]

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:405: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:427: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


array([ 1.    ,  0.    ,  3.    ,  1.    , 34.5   ,  0.    ,  0.    ,
        7.8292])

In [185]:
X.shape

(891, 8)

In [186]:
X_test.shape

(418, 8)

In [187]:
y_pred = rfr.predict(X_test)

In [192]:
subimssion_dict = {'PassengerId' : passenger_id, 'Survived' : y_pred}

In [193]:
submission = pd.DataFrame(subimssion_dict)

In [197]:
submission.tail()

,PassengerId,Survived
413,1305,0
414,1306,1
415,1307,0
416,1308,0
417,1309,1
